# Functions, Modules, Downloads and Package Installations



In [ ]:
# @title Package Installation
%pip install transformers datasets tokenizers nltk seqeval -q
%pip install accelerate -U
%pip install numpy matplotlib seaborn scikit-learn

In [ ]:
# @title Imports
import datasets
import numpy as np
import json
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import pipeline
from transformers import TrainingArguments, Trainer
from datasets import load_dataset, DatasetDict
from datasets.utils.logging import disable_progress_bar
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords, wordnet
import random
import re
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# @title Dataset & Corpus Downloads

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

dataset = datasets.load_dataset("surrey-nlp/PLOD-CW")

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

data_collator = DataCollatorForTokenClassification(tokenizer)

metric = datasets.load_metric("seqeval")

model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased",
                                                        num_labels = 4)

In [ ]:
# @title Hyperparameters & Configs

# DEFINITIONS

# List of tags in train/test sets
LABEL_LIST = sorted(["B-O", "B-AC", "B-LF", "I-LF"])

# Dynamically create a 0 - n mapping list for the tags
NER_MAPPING = {
    label: i for i, label in enumerate(LABEL_LIST)
}

# Remove the progress bars (removes bloat from outputs).
disable_progress_bar()

# Configs and their reset functions to reset their values after
# experiments.
def reset_configs():
    global MODEL_CONFIG, PREPROCESSING_CONFIG
    MODEL_CONFIG = {
        "learning_rate": 0.00001,
        "num_epochs": 3,
        "weight_decay": 0.01,
        "batch_size": 16
    }

    PREPROCESSING_CONFIG = {
        "apply_lemmatization": False,
        "apply_stemming": False,
        "apply_random_deletion": False,
        "apply_lowercasing": False,
        "apply_url_removal": False,
        "apply_stop_word_removal": False
    }

def reset_augments():
    global AUGMENTATION_CONFIG
    AUGMENTATION_CONFIG = {
        "apply_synonym_replacement": False,
        "apply_random_insertion": False,
        "apply_random_swap": False
    }

# Run once so these configs actually exist
reset_configs()
reset_augments()

In [ ]:
# @title Pre-Processing

# Handle data preprocessing according to config, can apply multiple different
# processes to text in sequence.

def preprocess_text(tokens, ner_tags, apply_lemmatization=False,
                    apply_stemming=False, apply_random_deletion=False,
                    apply_lowercasing=False, apply_url_removal=False,
                    apply_stop_word_removal=False):

    processed_tokens = []
    processed_tags = []

    for sentence_tokens, sentence_tags in zip(tokens, ner_tags):
        sentence_processed_tokens = []
        sentence_processed_tags = []

        # Iterate through token/tag pairs in the sequence
        for token, tag in zip(sentence_tokens, sentence_tags):

            # URL removal using a simple regex
            if apply_url_removal:
                token = re.sub(r'http\S+', '', token)

            # Lowercasing
            if apply_lowercasing:
                token = token.lower()

            # Lemmatization
            if apply_lemmatization:
                token = lemmatizer.lemmatize(token)

            # Stemming
            if apply_stemming:
                token = stemmer.stem(token)

            # Stop word removal, skips the iteration without appending the
            # current token/tag pair to "delete" them.
            if apply_stop_word_removal and token in stop_words:
                continue

            # Random deletion, randomly skips iterations with a 7.5% chance.
            if apply_random_deletion and random.random() < 0.075:
                continue

            # Append the processed token/tag pair to form a sentence.
            sentence_processed_tokens.append(token)
            sentence_processed_tags.append(tag)

        # Append the sentence to a list of sequences.
        processed_tokens.append(sentence_processed_tokens)
        processed_tags.append(sentence_processed_tags)

    return processed_tokens, processed_tags


# Extracts the token and tags split from the dataset to be passed to the text
# processor.

def preprocess_dataset(example, **preprocess_settings):
    return preprocess_text(example['tokens'],
                           example['ner_tags'],
                           **preprocess_settings)



In [ ]:
# @title Data Augmentation

def synonym_replacement(token):

    # Get all synoyms associated with token.
    synsets = wordnet.synsets(token)
    if not synsets:

        # Return the original token if not found.
        return token

    # Collect all synonyms from the synsets
    synonyms = [lem.name().replace('_', ' ') for
                syn in synsets for lem in syn.lemmas()]
    return random.choice(synonyms) if synonyms else token

def augment_sentence(tokens, ner_tags, apply_synonym_replacement=False,
                     apply_random_insertion=False, apply_random_swap=False):

    # Copy tokens/tags list to avoid modifying the original.
    new_tokens = tokens[:]
    new_tags = ner_tags[:]

    # Randomly swap two tokens and their corresponding tags.
    if apply_random_swap:
        idx1, idx2 = random.sample(range(len(new_tokens)), 2)
        new_tokens[idx1], new_tokens[idx2] = new_tokens[idx2], new_tokens[idx1]
        new_tags[idx1], new_tags[idx2] = new_tags[idx2], new_tags[idx1]

    # Replace each token with a synonym if possible.
    if apply_synonym_replacement:
        new_tokens = [synonym_replacement(token) for token in new_tokens]

    # Insert a similar word to a random token right after the original token.
    if apply_random_insertion:
        idx = random.randint(0, len(new_tokens) - 1)
        token, tag = new_tokens[idx], new_tags[idx]
        synonyms = [lem.name().replace('_', ' ') for syn
                    in wordnet.synsets(token) for lem in syn.lemmas()]
        if synonyms:
            synonym = random.choice(synonyms)
            new_tokens.insert(idx + 1, synonym)
            new_tags.insert(idx + 1, tag)

    return new_tokens, new_tags

def augment_dataset(dataset, **augmentation_config):
    augmented_tokens = []
    augmented_tags = []
    # Apply augmentations to each sequence in the dataset.
    for i, seq in enumerate(dataset["tokens"]):
        tokens, tags = augment_sentence(dataset["tokens"][i],
                                        dataset["ner_tags"][i],
                                        **augmentation_config)
        augmented_tokens.append(tokens)
        augmented_tags.append(tags)
    return augmented_tokens, augmented_tags



In [ ]:
# @title Data Processing Handler

# These functions cleanly handle all the relevant calls for augmentation,
# preprocessing and mapping of tags, and ensures theyre done in a logical order.

def ner_to_numerical(example):

    # maps the ner_tags to a numerical representation that the
    # transformer can interpret.
    numerical_tag_set = []
    for i, seq in enumerate(example["ner_tags"]):
        numerical_tags = []
        for tag in seq:
            numerical_tags.append(NER_MAPPING[tag])
        numerical_tag_set.append(numerical_tags)
    return numerical_tag_set

def preprocessing_tasks(example):

    # Augment data first, Preprocessing after and finally
    # map the ner_tags to numerical.
    example["tokens"], example["ner_tags"] = augment_dataset(
        example, **AUGMENTATION_CONFIG)
    example["tokens"], example["ner_tags"] = preprocess_dataset(
        example, **PREPROCESSING_CONFIG)
    example["ner_tags"] = ner_to_numerical(example)

    return example

def preprocessing_test_tasks(example):
    global PREPROCESSING_CONFIG

    # Only deterministic preprocessing should be applied to the test dataset.
    if PREPROCESSING_CONFIG["apply_random_deletion"]:

        PREPROCESSING_CONFIG["apply_random_deletion"] = False

        example["tokens"], example["ner_tags"] = preprocess_dataset(
            example, **PREPROCESSING_CONFIG)

        PREPROCESSING_CONFIG = True
    else:
        example["tokens"], example["ner_tags"] = preprocess_dataset(
            example, **PREPROCESSING_CONFIG)

    # Processing for the test dataset. We dont augment the data.
    example["ner_tags"] = ner_to_numerical(example)

    return example


In [ ]:
# @title Data Tokenizer and Token Padding Handler

# Tokenize data and handle the extra tokens added to beginning and end of
# sequences.

def label_tokenize_alignment(example, label_all_tokens = True):
    tokenized_input = tokenizer(example["tokens"],
                                truncation= True, is_split_into_words=True)
    labels = []

    # Process each sequence.
    for i, label in enumerate(example["ner_tags"]):

        # Get the token indices for the current sequence.
        word_ids = tokenized_input.word_ids(batch_index=i)
        previous_word_idx = None

        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:

                # These labels will ignored during training/evaluation.
                # Used to replace the start/end of sequences where the
                # tokenizer adds its own tokens.
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_input["labels"] = labels
    return tokenized_input

In [ ]:
# @title Testing, Validation and Evaluation

# Metrics to be run during training, evaluation and testing. Can be configured
# to return labels to feed into a confusion matrix.

def compute_metrics(eval_preds, return_labels = False):

    # get logits predictions during evaluation
    pred_logits, labels = eval_preds
    pred_logits = np.argmax(pred_logits, axis=2)

    # We ignore all the [token, tag] pairs where the tag is -100
    predictions = [[LABEL_LIST[eval_preds] for (eval_preds, l) in
                    zip(prediction, label) if l != -100]
                   for prediction, label in zip(pred_logits, labels)
    ]
    true_labels = [[LABEL_LIST[l] for (eval_preds, l) in
                    zip(prediction, label)if l != -100]
                   for prediction, label in zip(pred_logits, labels)
    ]

    # Calculate standard metrics for the experiments using the predicted and
    # true labels.
    results = metric.compute(predictions=predictions, references=true_labels)

    # Flatten the 2D lists for confusion matrix computation
    flattened_predictions = [p for sublist in predictions for p in sublist]
    flattened_true_labels = [t for sublist in true_labels for t in sublist]

    # Where the labels are needed, we return the flattened labels or
    # the standard metrics for these experiments.
    if return_labels:
        return {
            "flattened_true_labels": flattened_true_labels,
            "flattened_predictions": flattened_predictions
        }
    else:
        return {
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
        }


# Calls processing and augmentation functions on the dataset based on config.
# Trains, evaluates and tests a pretrained transformer model to predict
# ner_tags. Returns true and predicted labels for confusion matrix generating.

def train_validate_evaluate():

    # Separate testing portion of the dataset to use map functions on the
    # training and validation sets without interfering with the testing set.
    test_set = DatasetDict({
        "test": dataset["test"]
    })
    train_valid_set = DatasetDict({
        "train": dataset["train"],
        "validation": dataset["validation"]
    })

    # Run preprocessing tasks for the dataset splits according to configs.
    processed_dataset = train_valid_set.map(preprocessing_tasks, batched=True)
    processed_testset = test_set.map(preprocessing_test_tasks, batched=True)

    # Tokenize data and handle the tokenizer added tokens to start and end
    # of each sequence.
    tokenized_dataset = processed_dataset.map(label_tokenize_alignment,
                                              batched=True)
    tokenized_testset = processed_testset.map(label_tokenize_alignment,
                                              batched=True)

    # Define arguments for training.
    args = TrainingArguments(
        "test-ner",
        evaluation_strategy = "epoch",
        learning_rate=MODEL_CONFIG["learning_rate"],
        per_device_train_batch_size=MODEL_CONFIG["batch_size"],
        per_device_eval_batch_size=MODEL_CONFIG["batch_size"],
        num_train_epochs=MODEL_CONFIG["num_epochs"],
        weight_decay=MODEL_CONFIG["weight_decay"],
    )

    # Define the trainer.
    trainer = Trainer(
        model,
        args,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset["validation"],
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    # Begin Training with train dataset and validation dataset.
    trainer.train()

    # Evaluate model using test dataset.
    test_results = trainer.evaluate(tokenized_testset["test"])
    # Output the metrics from the model test.
    print(f"""\n    Loss: {test_results["eval_loss"]:.4f}
    F1 Score: {test_results["eval_f1"]:.4f}
    Accuracy: {test_results["eval_accuracy"]:.4f}\n""")
    print(f"    Evaluation time taken: {test_results['eval_runtime']:.4f}")

    # Set the model to predict tokens based on the test set.
    test_preds = trainer.predict(tokenized_testset["test"])

    # Compute metrics as before, except we only care about the predictions
    # and the true labels hence we use return_labels = True
    metric_results = compute_metrics((test_preds.predictions,
                                      tokenized_testset["test"]["ner_tags"]),
                                      return_labels = True)

    # Return the true labels and predicted labels for the confusion matrix
    return(
        metric_results["flattened_predictions"],
        metric_results["flattened_true_labels"]
    )

In [ ]:
# @title Confusion Matrix generator

# Generates a confusion matrix using true labels and predictions from model
# evaluation, it then plots it for manual analysis.

def confusion_matrix_gen(true_labels, predictions, matrix_title):
    conf_matrix = confusion_matrix(true_labels, predictions)

    plt.figure(figsize=(5, 3))
    sns.heatmap(conf_matrix, annot=True, fmt="g",
                cmap="Blues", xticklabels=LABEL_LIST, yticklabels=LABEL_LIST)
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.title(matrix_title)
    plt.show()

In [ ]:
# @title Configuration Setup

# A single function to encapsulate the whole process of running an experiment.
# Runs preprocessing, data augmentation, ner_tag mapping, model training,
# model evaluation, model testing and confusion matrix plotting.

def experiment_variation(matrix_title = "Confusion Matrix",
                         preprocess = [], augments = [], **kwargs):
    # Reset the configs from any previous experiments, simply ensures only the
    # chosen settings apply for this run.
    reset_configs()
    reset_augments()

    # Checks for all the model settings parsed into this function.
    for param, value in kwargs.items():
        MODEL_CONFIG[param] = value

    # Checks for all the preprocessing settings parsed into this function.
    for task in preprocess:
        PREPROCESSING_CONFIG[task] = True

    # Checks for all the data augmentation settings parsed into this function.
    for task in augments:
        AUGMENTATION_CONFIG[task] = True

    # Get flattened prediction and true labels from test dataset evaluation,
    # also runs the training, validation and evaluation of the model.
    flat_pred, flat_true = train_validate_evaluate()

    # Generate a confusion matrix using the flattened predictions and labels.
    confusion_matrix_gen(flat_true, flat_pred, matrix_title)



Using the experiment_variation function:

parameters:

        matrix_title   = type:[str]
        preprocess     = type:[list[str]]
        augments       = type:[list[str]]
        learning_rate  = type:[float]
        num_epochs     = type:[int]
        weight_decay   = type:[float]
        batch_size     = type:[int]

For preprocess you have thse options, pick as many as you want:

        apply_lemmatization
        apply_stemming
        apply_random_deletion
        apply_lowercasing
        apply_url_removal
        apply_stop_word_removal


For augments you have thse options, pick as many as you want:

        apply_synonym_replacement
        apply_random_insertion
        apply_random_swap

Example function usage:
        
        # Running back to back experiments without resetting environment will result in altered results.
        experiment_variation(
            matrix_title = "Experiment X",
            preprocess = [],
            augments = [],
            learning_rate = 0.00001,
            num_epochs = 1,
            weight_decay = 0.01,
            batch_size = 16
        )

# Experiment 1

Hyperparameter Testing


In [ ]:
# @title Testing 1 [Num_Epochs = 3]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 1 [Num_Epochs = 3]",
    num_epochs = 3,
)

In [ ]:
# @title Testing 2 [Num_Epochs = 5]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 2 [Num_Epochs = 5]",
    num_epochs = 5
)

In [ ]:
# @title Testing 3 [Num_Epochs = 7]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 3 [Num_Epochs = 7]",
    num_epochs = 7
)

In [ ]:
# @title Testing 4 [Learning_Rate = 0.000005]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 4 [Learning_Rate = 0.000005]",
    learning_rate = 0.000005,
    num_epochs = 7
)

In [ ]:
# @title Testing 5 [Learning_Rate = 0.00001]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 5 [Learning_Rate = 0.00001]",
    learning_rate = 0.00001,
    num_epochs = 7
)

In [ ]:
# @title Testing 6 [Learning_Rate = 0.00002]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 6 [Learning_Rate = 0.00002]",
    learning_rate = 0.00002,
    num_epochs = 7
)

In [ ]:
# @title Testing 7 [Batch_Size = 8]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 7 [Batch_Size = 8]",
    batch_size = 8,
    learning_rate = 0.00002,
    num_epochs = 7
)

In [ ]:
# @title Testing 8 [Batch_Size = 16]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 8 [Batch_Size = 16]",
    batch_size = 16,
    learning_rate = 0.00002,
    num_epochs = 7
)

In [ ]:
# @title Testing 9 [Batch_Size = 32]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 9 [Batch_Size = 32]",
    batch_size = 32,
    learning_rate = 0.00002,
    num_epochs = 7
)

In [ ]:
# @title Testing 10 [Weight_Decay = 0.04]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 10 [Weight_Decay = 0.04]",
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 11 [Weight_Decay = 0.08]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 11 [Weight_Decay = 0.08]",
    weight_decay = 0.08,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 12 [Weight_Decay = 0.12]

experiment_variation(
    matrix_title = "Experiment 1 - Testing 12 [Weight_Decay = 0.12]",
    weight_decay = 0.12,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

# Experiment 2

Preprocessing Testing

In [ ]:
# @title Testing 1 [Lemmatization]

experiment_variation(
    matrix_title = "Experiment 2 - Testing 1 [Lemmatization]",
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 2 [Stemming]

experiment_variation(
    matrix_title = "Experiment 2 - Testing 2 [Stemming]",
    preprocess = ["apply_stemming"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)


In [ ]:
# @title Testing 3 [Random Deletion]

experiment_variation(
    matrix_title = "Experiment 2 - Testing 3 [Random Deletion]",
    preprocess = ["apply_random_deletion"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)


In [ ]:
# @title Testing 4 [Lowercasing]

experiment_variation(
    matrix_title = "Experiment 2 - Testing 4 [Lowercasing]",
    preprocess = ["apply_lowercasing"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 5 [Url Removal]

experiment_variation(
    matrix_title = "Experiment 2 - Testing 5 [Url Removal]",
    preprocess = ["apply_url_removal"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 6 [Stop Word]

experiment_variation(
    matrix_title = "Experiment 2 - Testing 6 [Stop Word]",
    preprocess = ["apply_stop_word_removal"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 7 [Random Deletion, Url Removal, Stop Word]

experiment_variation(
    matrix_title = "Experiment 2 - Testing 7 [Random Deletion, Url Removal, Stop Word]",
    preprocess = ["apply_random_deletion", "apply_url_removal",
                  "apply_stop_word_removal"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 8 [Random Deletion, Url Removal, Stop Word, Lowercasing, Lemmatiziation]

experiment_variation(
    matrix_title = "Experiment 2 - Testing 8 [Random Deletion, Url Removal, Stop Word, Lowercasing, Lemmatiziation]",
    preprocess = ["apply_random_deletion", "apply_url_removal",
                  "apply_stop_word_removal", "apply_lowercasing",
                  "apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 9 [Random Deletion, Url Removal, Stop Word, Lowercasing, Stemming]

experiment_variation(
    matrix_title = "Experiment 2 - Testing 9 [Random Deletion, Url Removal, Stop Word, Lowercasing, Stemming]",
    preprocess = ["apply_random_deletion", "apply_url_removal",
                  "apply_stop_word_removal", "apply_lowercasing",
                  "apply_stemming"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

# Experiment 3



In [ ]:
# @title Testing 1 [Synonym Replacement]

experiment_variation(
    matrix_title = "Experiment 3 - Testing 1 [Synonym Replacement]",
    augments = ["apply_synonym_replacement"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 2 [Random Insertion]

experiment_variation(
    matrix_title = "Experiment 3 - Testing 2 [Random Insertion]",
    augments = ["apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 3 [Random Swapping]

experiment_variation(
    matrix_title = "Experiment 3 - Testing 3 [Random Swapping]",
    augments = ["apply_random_swap"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 4 [Synonym Replacement, Random Insertion]

experiment_variation(
    matrix_title = "Experiment 3 - Testing 4 [Synonym Replacement, Random Insertion]",
    augments = ["apply_synonym_replacement", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 5 [Synonym Replacement, Random Swapping]

experiment_variation(
    matrix_title = "Experiment 3 - Testing 5 [Synonym Replacement, Random Swapping]",
    augments = ["apply_synonym_replacement", "apply_random_swap"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 6 [Random Swapping, Random Insertion]

experiment_variation(
    matrix_title = "Experiment 3 -Testing 6 [Random Swapping, Random Insertion]",
    augments = ["apply_random_swap", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 7 [Synonym Replacement, Random Swapping, Random Insertion]

experiment_variation(
    matrix_title = "Experiment 3 - Testing 7 [Synonym Replacement, Random Swapping, Random Insertion]",
    augments = ["apply_synonym_replacement", "apply_random_swap",
                "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 16
)

# Experiment 4


In [ ]:
# @title Testing 1 - Increasing Num_Epochs to 9

experiment_variation(
    matrix_title = "Experiment 4 - Increasing Num_Epochs to 9",
    augments = ["apply_random_swap", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 9,
    batch_size = 16
)

In [ ]:
# @title Testing 2 - Increasing Num_Epochs to 8

experiment_variation(
    matrix_title = "Experiment 4 - Increasing Num_Epochs to 8",
    augments = ["apply_random_swap", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 8,
    batch_size = 16
)

In [ ]:
# @title Testing 3 - Decreasing Batch_Size to 8

experiment_variation(
    matrix_title = "Experiment 4 - Decreasing Batch_Size to 8",
    augments = ["apply_random_swap", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00002,
    num_epochs = 7,
    batch_size = 8
)

In [ ]:
# @title Testing 4 - Increasing Learning_Rate to 0.00003

experiment_variation(
    matrix_title = "Experiment 4 - Increasing Learning_Rate to 0.00003",
    augments = ["apply_random_swap", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.00003,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 5 - Increasing Learning_Rate to 0.000035

experiment_variation(
    matrix_title = "Experiment 4 - Increasing Learning_Rate to 0.000035",
    augments = ["apply_random_swap", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.000035,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 6 - Increasing Learning_Rate to 0.0000325

experiment_variation(
    matrix_title = "Experiment 4 - Increasing Learning_Rate to 0.0000325",
    augments = ["apply_random_swap", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.0000325,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 7 - Increasing Learning_Rate to 0.0000275

experiment_variation(
    matrix_title = "Experiment 4 - Increasing Learning_Rate to 0.0000275",
    augments = ["apply_random_swap", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.04,
    learning_rate = 0.0000275,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 8 - Decreasing Weight Decay To 0.03

experiment_variation(
    matrix_title = "Experiment 4 - Decreasing Weight Decay To 0.03",
    augments = ["apply_random_swap", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.03,
    learning_rate = 0.00003,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 9 - Decreasing Weight Decay To 0.025

experiment_variation(
    matrix_title = "Experiment 4 - Decreasing Weight Decay To 0.025",
    augments = ["apply_random_swap", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.025,
    learning_rate = 0.00003,
    num_epochs = 7,
    batch_size = 16
)

In [ ]:
# @title Testing 10 - Decreasing Weight Decay To 0.035

experiment_variation(
    matrix_title = "Experiment 4 - Decreasing Weight Decay To 0.035",
    augments = ["apply_random_swap", "apply_random_insertion"],
    preprocess = ["apply_lemmatization"],
    weight_decay = 0.035,
    learning_rate = 0.00003,
    num_epochs = 7,
    batch_size = 16
)

Best case parameters:
        
        experiment_variation(
            matrix_title = " ",
            augments = ["apply_random_swap", "apply_random_insertion"],
            preprocess = ["apply_lemmatization"],
            weight_decay = 0.03,
            learning_rate = 0.00003,
            num_epochs = 7,
            batch_size = 16
        )